## IMPORT LIBRARY & REFORAMT DATA


In [1]:
import os
import pandas as pd
from IPython.display import HTML, display

year = "2025"
month = "T7"
month_number = 7 

# let/data/2025/T2/data.xlsx
# Load the DataFrame
df = pd.read_excel(
    '../let/data/' + year + "/" + month + '/data.xlsx', 
    sheet_name='July 2025',
    usecols=[
    "Response ID",
    "Personal information []",
    "Personal information [].1",
    "Personal information [].2",
    "Personal information [].3",
    "Your English level based on ETOP (if applicable) or self-assessment:",
    "Date of joining Learn English Together"
    ],
)

# Rename columns
df.rename(columns={
    "Response ID": "RESPONSE_ID",
    "Personal information [].1": "PALS_ID",
    "Personal information []": "FULLNAME",   
    "Personal information [].2": "GRADE",
    "Personal information [].3": "SCHOOL",
    "Your English level based on ETOP (if applicable) or self-assessment:": "LEVEL",
    "Date of joining Learn English Together": "DATE_JOINED"
}, inplace=True)


df['PALS_ID'] = df['PALS_ID'].str.upper();

df.sort_values(by=['PALS_ID', 'DATE_JOINED'])
df['FULLNAME'] = df['FULLNAME'].str.upper()
print("TOTAL RECORDS: ", len(df), " RECORDS")
# Display the DataFrame as HTML
HTML(df.head().to_html())


TOTAL RECORDS:  846  RECORDS


,RESPONSE_ID,FULLNAME,PALS_ID,GRADE,SCHOOL,LEVEL,DATE_JOINED
0,56570,LÊ THỊ YẾN NHI,QB25G0637,8,THCS Sen Thuỷ,Beginner,2025-07-08 00:00:00
1,56573,ĐOÀN QUỐC THỊNH,DN25G0550,9,THCS NGUYỄN THIỆN THUẬT,Beginner,2025-07-08 00:00:00
2,56578,VÕ THỊ DIỄM HẰNG,QN25G0877,8,THCS NGUYỄN BỈNH KHIÊM,Beginner,2025-07-08 00:00:00
3,56580,NGUYỄN THANH NHẬT TRƯỜNG,HU25P0586,9/4,THCS Nguyễn Cư Trinh,Beginner,2025-07-08 00:00:00
4,56582,PHAN QUỲNH NHI,DN25G0555,8.9,THCS Nguyễn Huệ,Beginner,2025-07-08 00:00:00


## CHECK DATE_JOINED BE FROM MONDAY TO THURSDAY


In [2]:
# Convert DATE_JOINED column to datetime format
df['DATE_JOINED'] = pd.to_datetime(df['DATE_JOINED'], errors='coerce')

# Filter rows where DATE_JOINED is not from Monday to Thursday
# 0 - Monday
# 1 - Tuesday
# 2 - Wenesday
# 3 - Thursday
invalid_dates = df[(df['DATE_JOINED'].dt.weekday > 3) | (df['DATE_JOINED'].isna())]

# Display invalid rows
print("INVALID DATE_JOINED (not Monday to Thursday):", len(invalid_dates), "RECORDS")
display(invalid_dates)

# Filter only valid dates (Monday to Thursday)
valid_data = df[df['DATE_JOINED'].dt.weekday <= 3].dropna(subset=['DATE_JOINED'])

# Display the filtered valid data
print("VALID DATE_JOINED (Monday to Thursday):", len(valid_data), "RECORDS")
# HTML(valid_data.head(10).to_html())

INVALID DATE_JOINED (not Monday to Thursday): 10 RECORDS


,RESPONSE_ID,FULLNAME,PALS_ID,GRADE,SCHOOL,LEVEL,DATE_JOINED
102,56731,HOÀNG ANH THƯ,DN23P0485,10,Thái Phiên,Beginner,2025-07-12
104,56737,HOÀNG ANH THƯ,DN23P0485,10,Thái Phiên,Beginner,2025-07-13
303,57019,HOÀNG ANH THƯ,DN23P0485,10,Thái Phiên,Beginner,2025-07-18
306,57028,HOÀNG ANH THƯ,DN23P0485,10,Thái Phiên,Beginner,2025-07-19
316,57042,HOÀNG ANH THƯ,DN23P0485,10,Thái Phiên,Beginner,2025-07-20
327,57066,VÕ THỊ DIỄM HẰNG,QN25G0877,8,THCS NGUYỄN BỈNH KHIÊM,Beginner,2025-08-02
543,57361,NGUYỄN THỊ MINH THÙY,QT23P0427,11,THPT QUẢNG TRỊ,Intermediate,2025-07-06
546,57374,NGUYỄN THỊ ÁNH QUYÊN,QN25G0906,8,Thcs võ thị sáu,Beginner,2025-07-25
777,57685,NGUYỄN THỊ KIM NGÂN,DT23P0785,12,THPT Chu Văn An,Beginner,2025-07-19
840,57764,NGUYỄN THỊ ÁNH QUYÊN,QN25G0906,8,Thcs võ thị sáu,Beginner,2025-07-12


VALID DATE_JOINED (Monday to Thursday): 836 RECORDS


In [3]:
# Filter data for year 2025 and month 6 (June)
filtered_data = valid_data[
  (valid_data['DATE_JOINED'].dt.year == 2025) &
  (valid_data['DATE_JOINED'].dt.month == month_number)
]

print(f"RECORDS FOR YEAR 2025, MONTH {month_number}: {len(filtered_data)} RECORDS")
print(f"Date range: {filtered_data['DATE_JOINED'].min()} to {filtered_data['DATE_JOINED'].max()}")

display(filtered_data.head())

# Update valid_data to use the filtered data for subsequent processing
valid_data = filtered_data.copy()


RECORDS FOR YEAR 2025, MONTH 7: 835 RECORDS
Date range: 2025-07-01 00:00:00 to 2025-07-31 00:00:00


,RESPONSE_ID,FULLNAME,PALS_ID,GRADE,SCHOOL,LEVEL,DATE_JOINED
0,56570,LÊ THỊ YẾN NHI,QB25G0637,8,THCS Sen Thuỷ,Beginner,2025-07-08
1,56573,ĐOÀN QUỐC THỊNH,DN25G0550,9,THCS NGUYỄN THIỆN THUẬT,Beginner,2025-07-08
2,56578,VÕ THỊ DIỄM HẰNG,QN25G0877,8,THCS NGUYỄN BỈNH KHIÊM,Beginner,2025-07-08
3,56580,NGUYỄN THANH NHẬT TRƯỜNG,HU25P0586,9/4,THCS Nguyễn Cư Trinh,Beginner,2025-07-08
4,56582,PHAN QUỲNH NHI,DN25G0555,8.9,THCS Nguyễn Huệ,Beginner,2025-07-08


## CHECK DUPLICATED RECORDS


In [4]:
# Identify duplicate rows based on DATE_JOINED and PALS_ID
duplicates = valid_data[valid_data.duplicated(subset=['DATE_JOINED', 'PALS_ID'], keep=False)]

# Display duplicate rows
print("DUPLICATED RECORDS: ", len(duplicates), "RECORDS")
display(duplicates)

# Filter out duplicate rows to keep only unique rows based on DATE_JOINED and PALS_ID
unique_data = valid_data.drop_duplicates(subset=['DATE_JOINED', 'PALS_ID'], keep='first')

# Display the unique data
print("VALID RECORDS: ", len(unique_data), "RECORDS")

# EXPORT DATA
unique_data.to_excel('./export/' + year + "/" + month + '/unique_records.xlsx')
print("EXPORT SUCCESSFULLY")
# HTML(unique_data.to_html())

DUPLICATED RECORDS:  39 RECORDS


,RESPONSE_ID,FULLNAME,PALS_ID,GRADE,SCHOOL,LEVEL,DATE_JOINED
10,56589,NGUYỄN THỊ HUYỀN,QT25P0489,8,TH&THCS Triệu Sơn,Intermediate,2025-07-08
27,56616,PHAN NGUYỄN MINH TÙNG,HU25P0580,9/2,THCS Thủy Phương,Intermediate,2025-07-08
52,56659,PHAN NGUYỄN MINH TÙNG,HU25P0580,9/2,THCS Thủy Phương,Intermediate,2025-07-08
95,56720,PHAN NGUYỄN MINH TÙNG,HU25P0580,9/2,THCS Thủy Phương,Intermediate,2025-07-08
103,56734,LÊ THỊ THÙY DUNG,PY25P0045,8,THCS NGUYỄN TẤT THÀNH,Beginner,2025-07-10
135,56776,NGUYỄN THỊ HUYỀN,QT25P0489,8,TH&THCS Triệu Sơn,Intermediate,2025-07-08
139,56783,PHAN NGUYỄN MINH TÙNG,HU25P0580,9/2,THCS Thủy Phương,Intermediate,2025-07-08
150,56800,DƯƠNG HOÀNG BẢO AN,QN25G0909,8,Huỳnh thúc kháng,Beginner,2025-07-14
184,56842,PHAN NGUYỄN MINH TÙNG,HU25P0580,9/2,THCS Thủy Phương,Intermediate,2025-07-08
196,56855,BÙI PHẠM TRÀ MY,QG23P0652,10A12,THPT Võ Nguyên Giáp,Intermediate,2025-07-15


VALID RECORDS:  813 RECORDS
EXPORT SUCCESSFULLY


## COMBINE HOURS & EXPORT DATA


In [5]:
# Chỉ giữ lại 2 cột PALS_ID và FULLNAME
unique_data = unique_data[['PALS_ID', 'FULLNAME']]

# Làm sạch dữ liệu trước khi group by
unique_data['PALS_ID'] = unique_data['PALS_ID'].str.strip().str.upper()
unique_data['FULLNAME'] = unique_data['FULLNAME'].str.strip().str.upper()

# # Sau đó group by lại
total_hours_df = unique_data.groupby(['PALS_ID', 'FULLNAME']).size().reset_index(name='TOTAL_HOURS')


print("TOTAL RECORDS:", len(total_hours_df), " records")

TOTAL RECORDS: 135  records


## UNVALID PALS_ID


In [6]:
# Kiểm tra PALS_ID có hợp lệ theo regex
pattern = r"[A-Z]{2}[0-9]{2}[A-Z]{1}[0-9]{4}"
invalid_pals_id_df = total_hours_df[~total_hours_df['PALS_ID'].str.fullmatch(pattern)]

# In các PALS_ID không hợp lệ
print("Invalid PALS_IDs:", len(invalid_pals_id_df), " records")
display(invalid_pals_id_df)

# Xuất các PALS_ID không hợp lệ ra file Excel
invalid_output_path = './export/'  + year + "/" + month + '/invalid_pals_id_output.xlsx'
invalid_pals_id_df.to_excel(invalid_output_path, index=False)
print(f"Invalid PALS_IDs exported to {invalid_output_path}")

Invalid PALS_IDs: 10  records


,PALS_ID,FULLNAME,TOTAL_HOURS
0,0797040225,HOÀNG THỊ XUÂN NHI,1
2,BD23P0354 MAI HUỲNH GIAO,MAI HUỲNH GIAO,1
14,BD5P0385,LÒ ĐẶNG MINH KỲ,4
22,DN25G055550,ĐOÀN QUỐC THỊNH,1
73,QN250856,NGUYỄN DƯƠNG GIA BẢO,1
78,QN25G0883_NGUYỄN THỊ DIỄM QUỲNH,NGUYỄN THỊ DIỄM QUỲNH,1
99,QN25G900,ĐẶNG LÊ MINH HUY,1
108,QN25P08671,NGUYỄN HUỲNH NHƯ Ý,1
115,QN25P861,LÊ THỊ THẢO NGUYÊN,1
134,QT2G0495,LÊ TIẾN ĐẠT,1


Invalid PALS_IDs exported to ./export/2025/T7/invalid_pals_id_output.xlsx


## FINAL DATA


In [7]:
# Xuất các hàng hợp lệ với PALS_ID đúng mẫu ra file Excel khác
# Lọc các PALS_ID hợp lệ: phải khớp hoàn toàn với pattern (không có ký tự thừa)
valid_pals_id_df = total_hours_df[total_hours_df['PALS_ID'].str.fullmatch(pattern)]

print("VALID FINAL DATA: ", len(valid_pals_id_df), " records")
display(valid_pals_id_df)

valid_output_path = './export/'  + year + "/" + month + '/valid_pals_id_output.xlsx'
valid_pals_id_df.to_excel(valid_output_path, index=False)
print(f"Valid PALS_IDs exported to {valid_output_path}")

VALID FINAL DATA:  125  records


,PALS_ID,FULLNAME,TOTAL_HOURS
1,BD23P0354,MAI HUỲNH GIAO,8
3,BD23P0355,HUỲNH THỊ BẢO NGỌC,17
4,BD25G0392,ĐINH CẨM VÂN,3
5,BD25P0384,ĐỖ NGỌC TÍNH,8
6,BD25P0385,LÒ ĐẶNG MINH KỲ,2
...,...,...,...
129,QT25P0488,PHẠM HỒNG PHÚC,1
130,QT25P0489,NGUYỄN THỊ HUYỀN,9
131,QT25P0490,NGUYỄN NGỌC BẢO TRÂN,14
132,QT25P0492,NGUYỄN MAI YẾN NHƯ,13


Valid PALS_IDs exported to ./export/2025/T7/valid_pals_id_output.xlsx
